# Toplogy of Deep Neural Networks

This notebook will show you how easy it is to use gdeep to reproduce the experiments of the paper *Topology of Deep Neural Networks*, by Naizat et. al.

In [1]:
%reload_ext autoreload
%autoreload 2

# deep learning
import torch
from torch.optim import Adam, SGD
import numpy as np
from torch import nn
from gdeep.models import FFNet
from gdeep.data import TorchDataLoader
from gdeep.pipeline import Pipeline
from torch import autograd  

# plot
import plotly.express as px
import pandas as pd
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter()

# ML
from sklearn.preprocessing import MinMaxScaler
from sklearn.datasets import make_blobs
from sklearn.metrics import pairwise_distances

# TDA
from gtda.homology import VietorisRipsPersistence
from gtda.plotting import plot_diagram


No TPUs...


In [6]:
dl = TorchDataLoader(name="EntangledTori")
dl_tr, dl_ts = dl.build_dataloaders(batch_size=32)

In [7]:
# train NN
model = FFNet(arch=[3,15,15,15,15,15,15,15,15,15,2])
print(model)
pipe = Pipeline(model, (dl_tr, dl_ts), nn.CrossEntropyLoss(), writer)
pipe.train(Adam, 20, False, {"lr":0.01}, {"batch_size":10})

FFNet(
  (linears): ModuleList(
    (0): Linear(in_features=3, out_features=15, bias=True)
    (1): Linear(in_features=15, out_features=15, bias=True)
    (2): Linear(in_features=15, out_features=15, bias=True)
    (3): Linear(in_features=15, out_features=15, bias=True)
    (4): Linear(in_features=15, out_features=15, bias=True)
    (5): Linear(in_features=15, out_features=15, bias=True)
    (6): Linear(in_features=15, out_features=15, bias=True)
    (7): Linear(in_features=15, out_features=15, bias=True)
    (8): Linear(in_features=15, out_features=15, bias=True)
    (9): Linear(in_features=15, out_features=2, bias=True)
  )
)
Epoch 1
-------------------------------
Epoch training loss: 0.693957 	Epoch training accuracy: 50.01%                                      800 ]                       
Time taken for this epoch: 94.00s
Learning rate value: 0.01000000
Validation results: 
 Accuracy: 50.10%,                 Avg loss: 0.693145 

Epoch 2
-------------------------------
Epoch traini

KeyboardInterrupt: 